In [113]:
#Lets import pandas
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import matplotlib.pyplot as plt

In [85]:
#Load the dataset

San_train = pd.read_csv(r'E:\Study\Data Challenges\kaggle\Santander\train\train.csv')
San_test = pd.read_csv(r'E:\Study\Data Challenges\kaggle\Santander\test\test.csv')

In [86]:
San_train.shape

(76020, 371)

In [87]:
San_train.columns

Index(['ID', 'var3', 'var15', 'imp_ent_var16_ult1', 'imp_op_var39_comer_ult1',
       'imp_op_var39_comer_ult3', 'imp_op_var40_comer_ult1',
       'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1',
       'imp_op_var40_efect_ult3',
       ...
       'saldo_medio_var33_hace2', 'saldo_medio_var33_hace3',
       'saldo_medio_var33_ult1', 'saldo_medio_var33_ult3',
       'saldo_medio_var44_hace2', 'saldo_medio_var44_hace3',
       'saldo_medio_var44_ult1', 'saldo_medio_var44_ult3', 'var38', 'TARGET'],
      dtype='object', length=371)

In [88]:
#to find columns with unique values
i=0
for col in San_train.columns:
    if len(np.unique(San_train[col].values))==76020:
        print("Column %s has all unique values" %(col))
        i=i+1
print("No of columns with all unique values are %s" %i)

#But we are not going to delet the id column

Column ID has all unique values
No of columns with all unique values are 1


In [89]:
drop_train_col = []
drop_test_col = []
for col in San_train.columns:
            if len(np.unique(San_train[col].values)) == 1:
                San_train = San_train.drop(col,1)
                San_test = San_test.drop(col,1)
                drop_train_col.append(col)
                drop_test_col.append(col)
                print("Column %s has a constant value" %(col))

Column ind_var2_0 has a constant value
Column ind_var2 has a constant value
Column ind_var27_0 has a constant value
Column ind_var28_0 has a constant value
Column ind_var28 has a constant value
Column ind_var27 has a constant value
Column ind_var41 has a constant value
Column ind_var46_0 has a constant value
Column ind_var46 has a constant value
Column num_var27_0 has a constant value
Column num_var28_0 has a constant value
Column num_var28 has a constant value
Column num_var27 has a constant value
Column num_var41 has a constant value
Column num_var46_0 has a constant value
Column num_var46 has a constant value
Column saldo_var28 has a constant value
Column saldo_var27 has a constant value
Column saldo_var41 has a constant value
Column saldo_var46 has a constant value
Column imp_amort_var18_hace3 has a constant value
Column imp_amort_var34_hace3 has a constant value
Column imp_reemb_var13_hace3 has a constant value
Column imp_reemb_var33_hace3 has a constant value
Column imp_trasp_var

In [82]:
San_train.shape

(76020, 337)

In [83]:
# Test data should contain 1 less column, as it wouldnt have the target variable 

San_test.shape

(75818, 337)

In [102]:
# to check for columns whihc are present in train but not in test
#Before doing so lets seperate our target varibale

Target = San_train['TARGET']

for col in San_train.columns:
    if col not in San_test.columns:
        San_train=San_train.drop(col,1)
        print("%s column is present in train but not present in test" %(col))
        drop_train_col.append(col)      

TARGET column is present in train but not present in test


In [105]:
#Similarly removing variables in test data which are not in train

for col in San_test.columns:
    if col not in San_train.columns:
        San_train=San_test.drop(col,1)
        print("%s column is present in train but not present in test" %(col))
        drop_test_col.append(col)

In [106]:
San_train.shape

(76020, 336)

In [107]:
San_test.shape

(75818, 336)

In [111]:
# To find if there are any missing values

San_train.isnull().sum().sum()

0

In [112]:
San_test.isnull().sum().sum()

0

In [116]:
for col in San_train.columns:
    plt.boxplot(San_train[col])

In [117]:
plt.boxplot(San_train[col])

{'boxes': [<matplotlib.lines.Line2D at 0x1558e684a20>],
 'caps': [<matplotlib.lines.Line2D at 0x1558b489f28>,
 'fliers': [<matplotlib.lines.Line2D at 0x1558b494828>],
 'means': [],
 'medians': [<matplotlib.lines.Line2D at 0x1558b48d940>],
 'whiskers': [<matplotlib.lines.Line2D at 0x1558b482f28>,
  <matplotlib.lines.Line2D at 0x1558b489828>]}

In [118]:
San_test.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,2,32,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,40532.10
1,5,2,35,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,45486.72
2,6,2,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,46993.95
3,7,2,24,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,187898.61
4,9,2,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,73649.73


In [119]:
from sklearn.ensemble import RandomForestClassifier 

In [123]:
forest = RandomForestClassifier(n_estimators = 100)

In [124]:
forest = forest.fit(San_train,Target)

In [126]:
output = forest.predict(San_test)

In [127]:
output

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [131]:
first_submission = pd.DataFrame({'ID':San_test['ID'],'TARGET':output})

In [135]:
import os
os.chdir(r'E:\Study\Data Challenges\kaggle\Santander') 

In [140]:
# UNder sampling to balance the dataset

San_train['TARGET']=Target

In [169]:
len(San_train[San_train['TARGET']==1])

73012

In [151]:

under_san_train = pd.concat([San_train[San_train['TARGET']==0].sample(n=3008,replace=True),San_train[San_train['TARGET']==1]])

In [155]:
Target_under = under_san_train['TARGET']
under_san_train['TARGET'] = None

In [156]:
under_san_train.columns

Index(['ID', 'var3', 'var15', 'imp_ent_var16_ult1', 'imp_op_var39_comer_ult1',
       'imp_op_var39_comer_ult3', 'imp_op_var40_comer_ult1',
       'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1',
       'imp_op_var40_efect_ult3',
       ...
       'saldo_medio_var33_hace2', 'saldo_medio_var33_hace3',
       'saldo_medio_var33_ult1', 'saldo_medio_var33_ult3',
       'saldo_medio_var44_hace2', 'saldo_medio_var44_hace3',
       'saldo_medio_var44_ult1', 'saldo_medio_var44_ult3', 'var38', 'TARGET'],
      dtype='object', length=337)

In [160]:
under_san_train = under_san_train.drop('TARGET',1)

In [161]:
under_san_train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
4512,9054,2,26,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,974236.530000
31471,62916,2,28,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,118791.570000
62458,124713,2,57,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,126729.810000
878,1735,2,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,117310.979016
56544,112811,2,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,99790.860000


In [162]:
forest = RandomForestClassifier(n_estimators = 100,max_depth=)
forest = forest.fit(under_san_train,Target_under)
output_under = forest.predict(San_test)

In [163]:
second_submission = pd.DataFrame({'ID':San_test['ID'],'TARGET':output_under})

In [167]:
second_submission.to_csv("second_submission.csv")

In [168]:
first_submission.to_csv("first_submission.csv")

In [170]:
### Over sampling


over_san_train = pd.concat([San_train[San_train['TARGET']==1].sample(n=73012,replace=True),San_train[San_train['TARGET']==0]])


In [172]:
len(over_san_train)

146024

In [174]:
Target_over = over_san_train['TARGET']
over_san_train = over_san_train.drop('TARGET',1)

In [175]:
forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(over_san_train,Target_over)
output_over = forest.predict(San_test)

In [176]:
third_submission = pd.DataFrame({'ID':San_test['ID'],'TARGET':output_over})

In [178]:
third_submission.to_csv("third_submission.csv")

In [179]:
#under sampling without replacement 


under_san_train = pd.concat([San_train[San_train['TARGET']==0].sample(n=3008,replace=False),San_train[San_train['TARGET']==1]])

Target_under = under_san_train['TARGET']
under_san_train = under_san_train.drop('TARGET',1)

In [197]:
forest = RandomForestClassifier(n_estimators = 500,max_features= 0)
forest = forest.fit(under_san_train,Target_under)
output_under = forest.predict(San_test)

In [198]:
fifth_submission = pd.DataFrame({'ID':San_test['ID'],'TARGET':output_under})

fifth_submission.to_csv("fifth_submission.csv")

In [199]:
fifth_submission.sum()

ID        5752677905
TARGET         20927
dtype: int64